In [1]:
import os
import glob
import findspark
findspark.init()
import time
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StringType, IntegerType, StructField, ArrayType, DateType
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
schema = StructType([StructField('id', IntegerType(), True)
                     ,StructField('fecha', DateType(), True)
                     ,StructField('tipo_elem', StringType(), True)
                     ,StructField('intensidad', IntegerType(), True)
                     ,StructField('ocupacion', IntegerType(), True)
                     ,StructField('carga', IntegerType(), True)
                     ,StructField('vmed', IntegerType(), True)
                     ,StructField('error', IntegerType(), True)
                     ,StructField('periodo_integracion', IntegerType(), True)])

In [3]:
## DATASET FINAL ##
csv_files = glob.glob('datos_trafico/*.{}'.format('csv'))
añosPandemia = spark.createDataFrame([], schema)
for file in csv_files:
            temp = spark.read.csv(file,header=True, inferSchema=True, sep=";")
            añosPandemia = añosPandemia.union(temp)
añosPandemia = añosPandemia.withColumn("fecha",to_timestamp(col("fecha")))

In [4]:
def media_intensidad(fecha1, fecha2, nombre):
    fase = añosPandemia.where((añosPandemia.fecha >= fecha1) & (añosPandemia.fecha <= fecha2))
    media = fase.groupBy("id").mean("intensidad")
    rango = media.withColumn("RANGO", lit(nombre))
    rango = rango.drop("id").groupby("RANGO").mean("avg(intensidad)")
    rango = rango.withColumnRenamed("avg(avg(intensidad))","MEDIA INTENSIDAD TRAFICO")
    return rango

In [5]:
rango1 = media_intensidad("2019-12-20","2020-03-10","Rango 1") #PRIMER RANGO - 20 Diciembre 2019 hasta 10 Marzo 2020
rango2 = media_intensidad("2020-03-14","2020-05-10","Rango 2") #SEGUNDO RANGO - 14 Marzo 2020 hasta 10 Mayo 2020
rango3 = media_intensidad("2020-05-11","2020-10-26","Rango 3") #TERCER RANGO - 11 Mayo 2020 hasta 26 Octubre 2020
rango4 = media_intensidad("2020-10-27","2021-05-08","Rango 4") #CUARTO RANGO - 27 Octubre 2020 hasta 8 Mayo 2021
rango5 = media_intensidad("2021-05-09","2021-12-12","Rango 5") #QUINTO RANGO - 9 Mayo 2021 hasta 12 Diciembre 2021
rango6 = media_intensidad("2021-12-13","2022-06-26","Rango 6") #SEXTO RANGO - 13 Diciembre 2021 hasta 26 Junio 2022
rango7 = media_intensidad("2022-06-27","2022-12-31","Rango 7") #SEPTIMO RANGO - 27 Junio 2022 hasta 31 Diciembre 2022

In [6]:
rangos_medias = rango1.union(rango2).union(rango3).union(rango4).union(rango5).union(rango6).union(rango7)

In [7]:
rangos_medias.show()

+-------+------------------------+
|  RANGO|MEDIA INTENSIDAD TRAFICO|
+-------+------------------------+
|Rango 1|       403.9810309658522|
|Rango 2|      103.65971343341077|
|Rango 3|        288.029068682581|
|Rango 4|       329.6692256373894|
|Rango 5|      362.67685728618875|
|Rango 6|      364.69273030969066|
|Rango 7|      352.66431178608815|
+-------+------------------------+

